# Notes 12/28/17
* WMO format for exchanging observations
* Old, before internet
* All satellite data arrives in BUFR
* Different format for each obs
* Table driven
* High level scripts from Clara Draper
    * JEDI_conv_bufr2nc_2D.py
    * JEDI_amsu_bufr2nc.v8.py
* iMac is little-endian and test data may be big-endian
    * This happened with GSI test case
    * Can workaround this using: gfortran -fconvert=big-endian
        * This tells compiler that unformatted files are big-endian

## py-ncepbufr
* GitHub: jswhit/py-ncepbufr
* f2py wrapper
* Old Fortran code base (F77)
    * object files get collected into libbufr.a archive file
* If make change, rebuild by:
    * rm -rf src/libbufr.a build ~/.local/*
    * python setup.py install --user --record files.txt > setup.log 2>&1 &
    * Unfortunately, this system does not track dependencies
* bufr instance is constructed in ncepbufr/__init__.py
    * import _bufrlib #this is the fortran library libbufr.a
    * _bufrlib.fortran_open() # this opens the BUFR file, routine fortran_open in fortran_open.f
    * _bufrlib.openbf() # this reads in the table information, routine openbf in openbf.f
    * Ie, each method of the _bufrlib instance is a routine with the same name whose source code is in a file with the same name
    * The methods defined in __init__.py are the ones called by the top level python script

### setup.py
* First section will build the libbufr.a file if missing
    * Does not rebuild if a source file changes
        * source is .f (F77) and .c
    * cd's to src directory and runs makebufrlib.sh
* Second section creates three python extensions
    * _bufrlib
        * f2py wrapper of the libbufr.a file
        * Uses src/_bufrlib.pyf
            * Created by f2py run outside of this system, and optionally modified by developer
    * _read_convobs
        * f2py wrapper of src_diag/readconvobs.f90
    * _read_satobs
        * f2py wrappers of src_diag/readsatobs.f90, src_diag/read_diag.f90
* Third section creates the ncepbufr python package
    * builds a .so file for each of the three extensions
    * builds two packages: ncepbufr and read_diag

### Debug
* Successfully built and installed ncepbufr python packages
    * Inside and outside container
    * Need to install anaconda in both before setup.py would work
        * This is way to get Distutils python package which is what setup.py uses
* Tried JEDI_*.py scripts and code quits during the openbf call
    * Message:
        At line 63 of ifxy.f
        Fortran runtime error: Bad value during integer read
        
        FUNCTION IFXY(ADSC)
        
        CHARACTER*6 ADSC
        
        READ(ADSC, '(I1,I2,I3)') IF,IX,IY
        
        ADSC holds digits representing the code to look up in the BUFR table
        
    * Got same error when running test in the ncepbufr repository
        cd test
        python test.py
    * Error persisted with both python 2.7 and python 3.6
        * anaconda is configured to be able to switch between python2.7 and python3.6
            * source activate python27 --> python 2.7
            * source deactivate --> python 3.6
* Added iostat parameter to the READ command and got 5010 for the error code
    * 5010 means bad data (redundant to the initial message)
* Tried several things inside the ifxy routine
    * MYADSC as a local variable, and put MYADSC into the READ statement
        * copy ADSC into MYADSC
        * set MYADSC to '063000'
    * Variations on the READ statement
        * READ(ADSC(1:6), '(I1,I2,I3)') IF,IX,IY
        * READ(ADSC(1:1), '(I1)') IF, ditto for IX and IY
    * Print out value of ADSC
    * ADSC had the expected value in it, but error persisted
* Tried writing a separate test program
    * ADSC = '063000' in main
    * call function name IFXY
        * READ(ADSC, '(I1,I2,I3)') IF,IX,IY
    * This worked as expected
* Seems like something is stomping on memory
    * Tried setting stack space to "unlimited" (ulimit -s unlimited), but error persisted


# Notes, 1/2/18

## py-ncepbufr
* Tried -DNORMAL in makebufr.sh, did not work
    * This makes the array sizes considerably smaller
* Tried putting MAXNC back to 300 in bufrlib.h, did not work
* Tried setting IPRT = 1 in openbf.f (to get more diagnostics), but this gets a seg fault in WRDLEN
    * Seg fault is during a WRITE(UNIT=ERRSTR, FMT='(A,I2)') command in wrdlen.f
    * CHARACTER\*128 ERRSTR
    * Interesting that read/write from/to strings is what breaks
* Tried linking Fortran code that links directly to the libbufr.a archive
    * Directly call IFXY() works
    * Directly call FORTRAN_OPEN() and OPENBF() works
    * The problem seems to be in the python <--> Fortran interface
* Tried python27
    * Python interface gets Seg fault
    * Fortran direct calls works
* Tried small f2py test case suggested at website:
    * https://sysbio.ioc.ee/projects/f2py2e/index.html#usage
    * This works

# Notes, 1/4/18
## Meeting with Jeff Whitaker
* Issues with gfortran
    * I'm using gfortran 7
    * Someone else had issue with gfortran 7 and got it to work by using gfortran 6
        * Not a very good solution, but useful for debugging
    * Clara got hers working with g95
* Try older compiler on my iMac
    * gfortran 6 or g95
    * macports
    * direct download of source code or binaries
* Strings are tricky
    * f2py creates a C wrapper around the Fortran code, then the .so file provides the C routines callable from Python
    * Maybe something is getting corrupted in the string ifxy() is trying to read
        * null byte or some non-visible character
        * Dump out hex pattern
        * Not sure about this though, since the issue is with a Fortran string literal passed to ifxy()
* If I fix a bug, send pull request to Jeff on GitHub
    * MAXNC mismatch between .h (C) and .PRM (Fortran) files
    * Fix for read/write failures

## MacPorts
* Installation of MacPorts itself
    * Instructions: https://guide.macports.org/#installing.shell
    * Source install
        curl -O https://distfiles.macports.org/MacPorts/MacPorts-2.4.2.tar.bz2  
        tar xvf MacPorts-2.4.2.tar.bz2  
        cd MacPorts-2.4.2  
        ./configure  
        make  
        sudo make install  
        edit ~/.bash_profile
            export PATH="/opt/local/bin:/opt/local/sbin:$PATH"  
            export MANPATH="/opt/local/share/man:$MANPATH"  
        restart in a new window
        sudo port selfupdate
* Install gfortran 6
    * Command: sudo port install gcc6
        * installs gfortran 6.4.0 in: /opt/local/bin/gfortran-mp-6
    * Command: sudo port install gcc7
        * installs gfortran 7.2.0 in: /opt/local/bin/gfortran-mp-7
* Select a version
    * Command: sudo port select --set gcc mp-gcc6
        * creates symlinks for gcc, g++, gcov, gfortran pointing to corresponding -mp-6 versions
    * Command: sudo port select --set gcc mp-gcc7
        * creates symlinks pointing to -mp-7 versions
    * Command: sudo port select --set gcc none
        * deletes symlinks

## Use gfortran 6 to build py-ncepbufr package
* Edit src/makebufrlib.sh
    Comment out FC and CC settings
* Edit setup.py  
    Add at beginning:  
        os.environ["CC"] = "gcc-mp-6"  
        os.environ["CXX"] = "gcc-mp-6"  
        os.environ["FC"] = "gfortran-mp-6"  
        os.environ["F77"] = "gfortran-mp-6"  
        os.environ["F90"] = "gfortran-mp-6"  
* Run  
        python setup.py install --user --record=files.txt > setup.log 2>&1 &  


# Notes, 1/8/18
* Further debug efforts on the read error
* Created fork from jswhit/py-ncepbufr repository
    * Created branch: read_error
    * This is for debugging the error that occurs in ifxy() during the read statement
    * Created debug directory to do work, so that git doesn't get all cluttered with test files
    * Created debug/src directory
        * ifxy.f: subroutine that does a read on its argurment (string of digits)
        * tryme.f: subroutine that calls ifxy()
        * tryme_main.f: main program that calls ifxy()
        * ifxy.f90: version of ifxy.f in Fortran 90 syntax
        * tryme.f90: version of tryme.f in Fortran 90 syntax
* In the forked repository, on the read_error branch, tried setup.py with consistent C and Fortran compilers
    * gcc7 (gfortran-mp-7, gcc-mp-7) -> fails
    * gcc6 (gfortran-mp-6, gcc-mp-6) -> works
* For experiments:
    * For dynamic libraries and bundles, need to compile with -fPIC (position independent code)
    * .so file is a "bundle" which cannot be linked, but can be read in as a module in Python
        * This is what setup.py, f2py, etc. create
        * gfortran -bundle -undefined dynamic_lookup
    * .a file is a static library which can be linked
        * gfortran -fPIC -c *.f # creates .o file
        * ar *.o
    * .dylib file is a shared library which can be linked
        * gfortran -fPIC -dynamiclib -o libifxy.dylib ifxy.f
* In the debug directory
    * Created link_to_fortran directory for testing code directly linked to a fortran main program
        * build_pgms script will create two binaries: 
            * tryme_static: calls ifxy() through a static library version
            * tryme_dynamic: calls ifxy() through a dynamic library vesion
        * These both work using gcc7
    * Created various modules using f2py
        * build_modules script will build four modules (bundles):
            * mymod_gcc6.cpython-36m-darwin.so: gcc6 compiled, f77 source
            * mymod90_gcc6.cpyton-36m-darwin.so: gcc6 compiled, f90 source
            * mymod_gcc7.cpython-36m-darwin.so: gcc7 compiled, f77 source
            * mymod90_gcc7.cpyton-36m-darwin.so: gcc7 compiled, f90 source
        * The two gcc6 modules work, and the two gcc7 modules fail
        * Tried gcc6, f77 source with the "only: tryme" clause on f2py
            * This creates an interface (fortran -> C -> python) only for the tryme routine
            * This failed too
        * Tried changing optimization level for compilers
            * f2py --opt='-O2' ...: fails for gcc7
            * f2py --opt='-O1' ...: fails for gcc7
            * f2py --opt='-O0' ...: fails for gcc7
            

# Notes, 1/9/18
* Tried building a python module from scratch (without f2py)
    * Put this in debug/hand_made_module
    * Script build_all
* This initially broke for gfortran-mp-7
* Discovered that the issue is the wrong libgfortran file was getting linked in
    * Anaconda installs a libgfortran.dynlib in ~/anaconda3/lib
        * Apparently, this is compatible with gcc6 but not gcc7
        * Explains why read/write were broken
    * distutils places the anaconda3/lib path in the loader step (and not the gcc7 lib path)
* Workaround is to override the link path
    * Works to export LDPATH
        * LDPATH="-bundle -undefined dynamic_lookup -L/opt/local/lib/gcc7"
        * the bundle and undefined options are needed since these go away when the user sets LDPATH
* Real fix is to install gcc (c, c++, fortran compilers) into anaconda and fix the broken link path
    * conda install -c anaconda gcc
    * This works on my iMac
* Made small change to py-ncepbufr makebufr.sh script so that the user could pass in settings for the C and Fortran compilers

# Notes, 1/10/18
* Created pull request for the change in makebufr.sh that allows FC and CC to be set in environment
* Tried building py-ncepbufr in virtual machine
    * Installed gcc in anaconda
    * build worked, but shared library file fails to load
    * Get error: cannot map zero-fill pages
* Search on net says this is due to running out of memory
    * Tried test program (in ~/project/test_alloc) which was able to allocate 4GB!
    * Not a memory issue, but I'm not able to find what else can cause the share library to not load


# Notes, 1/25/18
* Help Xin with converting aircraft obs from BUFR to netCDF
* He isn't quite ready yet to use the netCDF aircraft obs data
* He perfers to use a utility (pb2nc) from the DTC MET group
* Easiest way to get access to pb2nc is to run MET's Docker image
    * Note: 6.1 is the latest version, but it is missing from the website
    
~~~~~~~~~
wget http://www.dtcenter.org/met/users/support/online_tutorial/docker_data/container-met-6.0-tutorial.tar.gz
    
docker load -i container-met-6.0-tutorial.tar.gz
mkdir MET_Tutorial_6.0
cd MET_Tutorial_6.0
export MET_TUTORIAL_DIR=$(pwd)

docker run --rm -it -v ${MET_TUTORIAL_DIR}/MET_Tutorial:/met/met-6.0/MET_Tutorial met-6.0-tutorial /bin/bash
cd /met/met-6.0
~~~~~~~~~

* Once inside the container (following commands above), try a couple conversions
    * Create a workspace: /met/met6.0/projects/pb2nc, and cd there
    * Tutorial example
        * wget https://dtcenter.org/met/users/support/online_tutorial/METv6.1/config/PB2NCConfig_default
        * mv PB2NCConfig_default PB2NCConfig_tutorial
        * Follow edit instructions:  
            https://dtcenter.org/met/users/support/online_tutorial/METv6.1/tutorial.php?name=pb2nc&category=configure
        * pb2nc ../../data/sample_obs/prepbufr/ndas.t00z.prepbufr.tm12.20070401.nr ndas_pb.nc PB2NCConfig_default
    * Xin's file example
        * wget http://www.ftp.ncep.noaa.gov/data/nccf/com/gfs/prod/gfs.2018012506/gfs.t06z.prepbufr.nr
        * cp PB2NCConfig_tutorial PB2NCConfig_aircraft
        * edit PB2NCConfig_aircraft
            * Change the message_type array to [ "AIRCFT", "AIRCAR" ]
            * Change obs_window to -5400, 5400 (3 hour window with time 0 in the center)
        * pb2nc gfs.t06z.prepbufr.nr gfs_pb.nc PB2NCConfig_aircraft
* Handy guide for BUFR  
    https://dtcenter.org/com-GSI/BUFR/docs/users_guide/BUFR_PrepBUFR_User_Guide_v1.pdf  
    
    * Put a copy of above guide on my UCAR Google Drive
    * Page 72 has links to handy tables showing many of the message codes

# Notes, 1/31/18
* The above docker command that runs the containter (docker run ...) sets up a shared directory between host and guest processes
    * -v host_path:guest_path
    * Any files placed by the host process into host_path will show up in the guest process in guest_path, and vice-versa
* Table showing meaning of BUFR mnemonics  
    http://www.emc.ncep.noaa.gov/mmb/data_processing/prepbufr.doc/table_1.htm
